In [ ]:
!pip install pymysql

In [ ]:
# 프론트엔드부 (수신)
import json
import pymysql

con = pymysql.connect(host='34.64.86.80', port = 3306, user = 'root', passwd = 'acorn1234',
                     use_unicode = True, db = 'mqtt', charset = 'utf8', autocommit=True)
cursor = con.cursor()

def on_connect(client, userdata, flags, rc):
  print("연결되었습니다 " + str(rc))
  client.subscribe('house/light')

def on_message(client, userdata, msg):
  json_return = json.loads(msg.payload.decode('utf-8'))  # 한글 표현
  print(json_return["light"])
  print(json_return["hangeul"])
  print(msg.topic + " " + str(json_return))
  try:
    with con.cursor() as cursor:
      sql = "insert into mqtt (light, door, close, hangeul) values (%s, %s, %s, %s)"
      cursor.execute(sql, (json_return['light'], json_return['door'], json_return['close'], json_return['hangeul']))
    con.commit()
    print('저장 완료')
  except Exception as e:
    print('실행 오류:', e)
  if json_return['close'] == 'true':
    client.disconnect()

def on_disconnect(client, userdata, rc):
  print("해제 이유: " + str(rc))
  con.close()
  client.connected_flag = False
  client.disconnect_flag = True

def on_subscribed(client, userdata, topic):
  print('다음 토픽에 등록됨 : $topic')

def on_subscribe(client, userdata, mid, granted_qos):
  print("subscribed: " + str(mid) + " " + str(granted_qos))

def on_close(client, userdata, rc):
  print("종료")
  con.close()

In [ ]:
client = mqtt.Client()
client.username_pw_set(username = 'mirae', password='daejeon1234')
client.on_connect = on_connect
client.on_message = on_message
client.on_disconnect = on_disconnect
client.on_subscribe = on_subscribe
client.on_subscribed = on_subscribed
client.on_close = on_close
client.connect('34.64.86.80', 19883)
client.subscribe('house/light', qos=1)
client.loop_forever()

연결되었습니다 0
subscribed: 1 (1,)
subscribed: 2 (0,)
on
대전
house/light {'light': 'on', 'door': 'close', 'close': 'false', 'hangeul': '대전'}
저장 완료
on
대전
house/light {'light': 'on', 'door': 'close', 'close': 'true', 'hangeul': '대전'}
저장 완료
해제 이유: 0


7

In [ ]:
# 백엔드부 (송신)
import json
message = {}
message["light"] = "on"
message["door"] = "close"
message["close"] = "false"
# message["close"] = "true"

message["hangeul"] = "대전"
transferdata = json.dumps(message)
client.publish("house/light", transferdata)

In [ ]:
# 직접 실습해본것

In [ ]:
import pymysql
import json
import paho.mqtt.client as mqtt
con = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234', 
                      db='mtqq', charset='utf8', autocommit=True)
message_count = 0
def on_connect(client, userdata, flags, rc):
    print('연결되었습니다' + str(rc))

def on_message(client, userdata, msg):
    global message_count  
    json_message = msg.payload.decode('utf-8')
    data = json.loads(json_message)

    light = data['light']
    door = data['door']
    
    cursor = con.cursor()
    query = "insert into mtqq (light, door) values (%s, %s)"
    values = (light, door)
    cursor.execute(query, values)
    con.commit()
    print('성공')
    cursor.close()

    message_count += 1
    if message_count >= 3:
      client.disconnect()  # Disconnect the MQTT client

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect('34.64.86.80', 19883)
client.subscribe('house/light', qos=1)
client.loop_forever() 
 
con.close() 

In [ ]:
#
import json
import paho.mqtt.client as mqtt
client = mqtt.Client()
client.connect('34.64.86.80', 19883) 
message = {}
message['light'] = 'off'
message['door'] = 'open'
transferdata = json.dumps(message)
client.publish('house/light', transferdata)